In [1]:
import sys
import numpy as np
import cv2
sys.path.append('..')    
from letter_extraction import ThresholdParams, ThresholdDetector, EdgeDetector, ChunkExtractor
from letter_extraction import ExtractFragmentParams, ContoursExtractor, CompositeExtractor
from metrics import KeyboardLayout, LetterExtractionMetric

In [2]:
layout_func = (lambda x: KeyboardLayout("../keyboards/layouts/keyboard{}.layout".format(x), '../'))
keyboard_layouts = map(layout_func, range(1, 26))

In [3]:
def test_contours_extractor(contours_detector, extract_params):
    contours_extractor = ContoursExtractor(contours_detector, extract_params)
    return LetterExtractionMetric.score_layouts(keyboard_layouts, contours_extractor)

def test_extract_params(contours_detector):
    max_score = 0.0
    best_params = None
    for min_scale in [50, 100]:
        for max_scale in [20, 10]:
            for extract_scale in [0.9, 1.0, 1.1, 1.2, 1.3, 1.5]:
                extract_params = ExtractFragmentParams(min_scale, max_scale, extract_scale)
                score = test_contours_extractor(contours_detector, extract_params)
                if score > max_score:
                    score = max_score
                    best_params = extract_params
    return {'score': max_score, 'params': best_params}

## Testing ChunkDetector params

In [9]:
def test_chunk_detector_params():
    max_score = 0.0
    for chunk_count in np.linspace(20, 80, 13):
        chunk_extractor = ChunkExtractor(chunk_count)
        score = LetterExtractionMetric.score_layouts(keyboard_layouts, chunk_extractor)
        print("Chunk count: {}, score: {}".format(chunk_count, score))

In [10]:
test_chunk_detector_params()

Chunk count: 20.0, score: 133.551175214
Chunk count: 25.0, score: 342.321594115
Chunk count: 30.0, score: 547.640452274
Chunk count: 35.0, score: 698.855332663
Chunk count: 40.0, score: 720.850525244
Chunk count: 45.0, score: 725.94133708
Chunk count: 50.0, score: 573.652235753
Chunk count: 55.0, score: 666.203172986
Chunk count: 60.0, score: 514.265862061
Chunk count: 65.0, score: 500.091657169
Chunk count: 70.0, score: 387.632321433
Chunk count: 75.0, score: 393.203271804
Chunk count: 80.0, score: 303.264436469


## Testing EdgeDetector params

In [11]:
def test_edge_detector_params():
    pass


In [12]:
test_edge_detector_params()

## Testing ThresholdDetector params

In [6]:
def test_thres_detector_params():
    max_score = 0.0
    for r_approx in [cv2.CHAIN_APPROX_NONE, cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_TC89_L1]:
        for r_type in [cv2.RETR_LIST, cv2.RETR_TREE, cv2.RETR_EXTERNAL]:
            for min_poly in [3, 5, 7]:
                for b_scale in [20, 30, 40]:
                    for s_thres in [-30, 0, 30]:                                                                        
                        thres_params = ThresholdParams(b_scale, s_thres, r_type, r_approx, min_poly)
                        contours_detector = ThresholdDetector(thres_params)
                        score = test_extract_params(contours_detector)['score']
                        if score > max_score:
                            max_score = score
                            best_params = thres_params
                        print("Params: {}, score: {} (best: {})".format(thres_params, score, max_score))
    return thres_params

In [ ]:
test_thres_detector_params()

Params: ThresholdParams(block_scale=20, sub_threshold=-30, retr_type=1L, retr_approx=1L, min_poly=3), score: 0.0 (best: 0.0)
Params: ThresholdParams(block_scale=20, sub_threshold=0, retr_type=1L, retr_approx=1L, min_poly=3), score: 0.0 (best: 0.0)
Params: ThresholdParams(block_scale=20, sub_threshold=30, retr_type=1L, retr_approx=1L, min_poly=3), score: 0.0 (best: 0.0)
Params: ThresholdParams(block_scale=30, sub_threshold=-30, retr_type=1L, retr_approx=1L, min_poly=3), score: 0.0 (best: 0.0)
